# One Shot

Ziel ist es, mit einem Request alle Antworten zu erhalten.

In [ ]:
import sys
sys.path.append('..')

from survey.models import StrategyModel, QuestionsModel, ModelsModel, ResponseModel
from survey.response import Requests

# strategie laden
strategies = StrategyModel()
strategies.load_strategies('oneshot_%')
strategies_list = strategies.get_strategies()

# fragen laden
questions = QuestionsModel()
questions_list = questions.get_questions()

# models laden
models = ModelsModel()
models_list = models.get_models()

# request und response laden
requests = Requests()
response = ResponseModel()

In [ ]:
print(f"Strategies: {len(strategies_list)} - {strategies_list}")
print(f"Questions: {len(questions_list)}")
print(f"Models: {len(models_list)}")

In [ ]:
for i, strategy in enumerate(strategies_list):

    # prompts lesen
    with open(f"../{strategies.systemprompt_path(i)}", "r") as f:
        system_prompt = f.read()

    with open(f"../{strategies.messageprompt_path(i)}", "r") as f:
        message_prompt = f.read()

    # prompt anreichern
    questions_text = "\n".join([f"{id}. {text}" for id, text in questions_list])
    message = message_prompt.replace("{questions}", questions_text)

    for model_db_id, model_id in models_list:

        answer, duration_time = requests.request_model(model_id, system_prompt, message)

        # DEBUG - Was kommt zurück?
        print(f"\n=== Model: {model_id} ===")
        print(answer.choices[0].message.content)
        print("=" * 50)

        response.write_respone(model_db_id, strategy['id'], duration_time, answer)